# DART Report Crawling

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import os

import OpenDartReader

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
dart = OpenDartReader(os.environ["DART_API_KEY"])

## 1. 공시검색 (모든종목)

- `start`와 `end`는 3개월 이내 범위만 가능함

In [6]:
# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (3,139 건)
df = dart.list(start="2022-01-01", end="2022-01-10")

In [7]:
df.columns

Index(['corp_code', 'corp_name', 'stock_code', 'corp_cls', 'report_nm',
       'rcept_no', 'flr_nm', 'rcept_dt', 'rm'],
      dtype='object')

In [9]:
rcept_no_list = df["rcept_no"].tolist()

## 2. 공시문서 크롤링

In [10]:
rcept_no = rcept_no_list[0]

xml_text = dart.document(rcept_no)

In [17]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(xml_text, "xml")
text_only = soup.get_text(strip=False)

In [18]:
print(text_only)


자산양도등의등록신청서
2.6
뉴스타해피제삼차유동화전문 유한회사



자산양도등의등록신청서











자산유동화관련 등록ㆍ신고등 현황 :



2022년 01월 07일 
[변경등록] 자산유동화계획의등록신청서[신탁회사[은행신탁계정포함)]


2021년 12월 08일 
자산양도등의등록신청서(유동화전문회사)


2021년 12월 07일 
[변경등록] 자산유동화계획의등록신청서[신탁회사[은행신탁계정포함)]


2021년 11월 08일 
자산양도등의등록신청서(유동화전문회사)


2021년 11월 05일 
[변경등록] 자산유동화계획의등록신청서[신탁회사[은행신탁계정포함)]


2021년 10월 12일 
자산양도등의등록신청서(유동화전문회사)


2021년 10월 08일 
[변경등록] 자산유동화계획의등록신청서[신탁회사[은행신탁계정포함)]


2021년 09월 08일 
자산양도등의등록신청서(유동화전문회사)


2021년 09월 07일

[변경등록] 자산유동화계획의등록신청서[신탁회사[은행신탁계정포함)]



2021년 08월 30일
자산양도등의등록신청서(유동화전문회사)


2021년 08월 30일
자산양도등의등록신청서 [신탁회사(은행신탁계정포함)]


2021년 08월 27일 
[변경등록]자산유동화계획의등록신청서(유동화전문회사)


2021년 08월 27일 
[변경등록]자산유동화계획의등록신청서[신탁회사(은행신탁계정포함)]


2021년 08월 12일
자산유동화계획의등록신청서 (유동화전문회사) 최초 제출


2021년 08월 12일
자산유동화계획의등록신청서 [신탁회사(은행신탁계정포함)] 최초 제출


금융위원회 귀중





자산유동화에관한법률 제6조의 규정에 의하여 자산양도등의 등록을 신청합니다.





2022년 01월 10일





신   청   인  



회   사   명 :
주식회사 다날


대 표 이 사 :
박 상 만 (인)


본점소재지 :
경기도 성남시 분당구 분당로 55, 9층(서현동, 분당퍼스트타워)


(전화번호) 031-697-1004


작성책임